## Job Recomendation Using Knowledge Graph

In [102]:
import pandas as pd
import numpy as np
from itertools import chain
from collections import Counter


#### Importing Dataset

In [103]:
jd = pd.read_csv("Job Description Dataset/dice_com-job_us_sample.csv")

In [104]:
dataset = pd.read_csv("Resume_Dataset/survey_results_public.csv")

C:\Users\hp\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (8,12,13,14,15,16,50,51,52,53,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128) have mixed types.Specify dtype option on import or set low_memory=False.
  from matplotlib_inline.backend_inline import configure_inline_support


## Data Cleaning
#### Creating resume subset using relavent dimensionality

In [105]:
resume_subset= dataset[['Respondent','Country','Student','CompanySize','Dependents','DevType','Gender','JobSatisfaction','LastNewJob','Salary','SalaryType','LanguageWorkedWith','LanguageDesireNextYear','DatabaseWorkedWith','DatabaseDesireNextYear','PlatformWorkedWith','PlatformDesireNextYear','FrameworkWorkedWith','FrameworkDesireNextYear','OperatingSystem','HypotheticalTools1','Age']]
df = pd.DataFrame(resume_subset)
df

,Respondent,Country,Student,CompanySize,Dependents,DevType,Gender,JobSatisfaction,LastNewJob,Salary,...,LanguageDesireNextYear,DatabaseWorkedWith,DatabaseDesireNextYear,PlatformWorkedWith,PlatformDesireNextYear,FrameworkWorkedWith,FrameworkDesireNextYear,OperatingSystem,HypotheticalTools1,Age
0,1,Kenya,No,20 to 99 employees,Yes,Full-stack developer,Male,Extremely satisfied,Less than a year ago,NaN,...,JavaScript;Python;HTML;CSS,Redis;SQL Server;MySQL;PostgreSQL;Amazon RDS/A...,Redis;SQL Server;MySQL;PostgreSQL;Amazon RDS/A...,AWS;Azure;Linux;Firebase,AWS;Azure;Linux;Firebase,Django;React,Django;React,Linux-based,Extremely interested,25 - 34 years old
1,3,United Kingdom,No,"10,000 or more employees",Yes,Database administrator;DevOps specialist;Full-...,Male,Moderately dissatisfied,More than 4 years ago,51000,...,Go;Python,Redis;PostgreSQL;Memcached,PostgreSQL,Linux,Linux,Django,React,Linux-based,A little bit interested,35 - 44 years old
2,4,United States,No,20 to 99 employees,NaN,Engineering manager;Full-stack developer,NaN,Moderately satisfied,Less than a year ago,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5,United States,No,100 to 499 employees,No,Full-stack developer,Male,Neither satisfied nor dissatisfied,Less than a year ago,NaN,...,C#;JavaScript;SQL;TypeScript;HTML;CSS;Bash/Shell,"SQL Server;Microsoft Azure (Tables, CosmosDB, ...","SQL Server;Microsoft Azure (Tables, CosmosDB, ...",Azure,Azure,NaN,Angular;.NET Core;React,Windows,Somewhat interested,35 - 44 years old
4,7,South Africa,"Yes, part-time","10,000 or more employees",Yes,Data or business analyst;Desktop or enterprise...,Male,Slightly satisfied,Between 1 and 2 years ago,260000,...,Assembly;C;C++;Matlab;SQL;Bash/Shell,SQL Server;PostgreSQL;Oracle;IBM Db2,PostgreSQL;Oracle;IBM Db2,Arduino;Windows Desktop or Server,Arduino;Windows Desktop or Server,NaN,NaN,Windows,Extremely interested,18 - 24 years old
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98850,101513,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98851,101531,Spain,"Yes, full-time",NaN,NaN,Back-end developer;Front-end developer,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98852,101541,India,"Yes, full-time",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98853,101544,Russian Federation,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Handling NA Values

In [106]:
resume_subset.isna().sum()

Respondent                     0
Country                      412
Student                     3954
CompanySize                27324
Dependents                 36259
DevType                     6757
Gender                     34386
JobSatisfaction            29579
LastNewJob                 19966
Salary                     48277
SalaryType                 47785
LanguageWorkedWith         20521
LanguageDesireNextYear     25611
DatabaseWorkedWith         32585
DatabaseDesireNextYear     41357
PlatformWorkedWith         32856
PlatformDesireNextYear     37502
FrameworkWorkedWith        47235
FrameworkDesireNextYear    43637
OperatingSystem            22676
HypotheticalTools1         28787
Age                        34281
dtype: int64

In [107]:
resume_subset.describe()

,Respondent
count,98855.000000
mean,50822.971635
std,29321.650410
min,1.000000
25%,25443.500000
50%,50823.000000
75%,76219.500000
max,101592.000000


In [108]:
dataset.columns

Index(['Respondent', 'Hobby', 'OpenSource', 'Country', 'Student', 'Employment',
       'FormalEducation', 'UndergradMajor', 'CompanySize', 'DevType',
       ...
       'Exercise', 'Gender', 'SexualOrientation', 'EducationParents',
       'RaceEthnicity', 'Age', 'Dependents', 'MilitaryUS', 'SurveyTooLong',
       'SurveyEasy'],
      dtype='object', length=129)

In [109]:
resume_subset.dtypes

Respondent                  int64
Country                    object
Student                    object
CompanySize                object
Dependents                 object
DevType                    object
Gender                     object
JobSatisfaction            object
LastNewJob                 object
Salary                     object
SalaryType                 object
LanguageWorkedWith         object
LanguageDesireNextYear     object
DatabaseWorkedWith         object
DatabaseDesireNextYear     object
PlatformWorkedWith         object
PlatformDesireNextYear     object
FrameworkWorkedWith        object
FrameworkDesireNextYear    object
OperatingSystem            object
HypotheticalTools1         object
Age                        object
dtype: object

In [110]:
Resume= resume_subset.dropna()
Resume

,Respondent,Country,Student,CompanySize,Dependents,DevType,Gender,JobSatisfaction,LastNewJob,Salary,...,LanguageDesireNextYear,DatabaseWorkedWith,DatabaseDesireNextYear,PlatformWorkedWith,PlatformDesireNextYear,FrameworkWorkedWith,FrameworkDesireNextYear,OperatingSystem,HypotheticalTools1,Age
1,3,United Kingdom,No,"10,000 or more employees",Yes,Database administrator;DevOps specialist;Full-...,Male,Moderately dissatisfied,More than 4 years ago,51000,...,Go;Python,Redis;PostgreSQL;Memcached,PostgreSQL,Linux,Linux,Django,React,Linux-based,A little bit interested,35 - 44 years old
17,27,Sweden,No,10 to 19 employees,Yes,Back-end developer,Male,Moderately satisfied,Between 1 and 2 years ago,32000,...,C#;F#;Haskell;SQL;Ocaml,SQL Server,Redis;SQL Server;PostgreSQL;Elasticsearch,Linux;Windows Desktop or Server,Linux;Windows Desktop or Server,.NET Core,.NET Core,Windows,Not at all interested,35 - 44 years old
20,33,Australia,No,"1,000 to 4,999 employees",Yes,Database administrator;Full-stack developer;Mo...,Male,Slightly satisfied,Between 2 and 4 years ago,120000,...,C;C++;Go;Python;SQL;Swift;Kotlin,Redis;PostgreSQL;SQLite,Redis;PostgreSQL;SQLite;Google BigQuery,Android;AWS;iOS;Linux;Mac OS;Firebase,Android;AWS;Google Cloud Platform/App Engine;i...,Angular;Node.js,Angular;Node.js;TensorFlow,MacOS,A little bit interested,35 - 44 years old
22,37,United Kingdom,No,20 to 99 employees,No,Back-end developer;Database administrator;Desi...,Male,Slightly satisfied,More than 4 years ago,25,...,C#;JavaScript;PHP;SQL;HTML;CSS,MySQL,SQL Server;MySQL,Linux;Mac OS;Windows Desktop or Server,Linux;Mac OS;Windows Desktop or Server,.NET Core,.NET Core,Windows,Somewhat interested,25 - 34 years old
24,39,India,No,100 to 499 employees,No,Mobile developer,Female;Male,Moderately satisfied,Between 1 and 2 years ago,700000,...,Java;JavaScript;Swift;Kotlin,SQL Server;MySQL,MySQL,Android;iOS;Firebase,Android;iOS;Firebase,Angular;Cordova,Cordova,MacOS,Somewhat interested,25 - 34 years old
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71503,101558,Germany,No,20 to 99 employees,No,Full-stack developer,Male,Slightly dissatisfied,Less than a year ago,50000,...,Ruby;Rust;Swift;HTML;CSS;Bash/Shell,MongoDB;Redis;MySQL;PostgreSQL;Microsoft Azure...,Redis;PostgreSQL;Elasticsearch,AWS;Azure;Heroku;iOS;Mac OS;SharePoint,Heroku;iOS;Mac OS,Angular;.NET Core;Cordova,React,MacOS,A little bit interested,25 - 34 years old
71504,101559,Italy,No,100 to 499 employees,No,Front-end developer;Game or graphics developer...,Male,Moderately satisfied,Less than a year ago,25000,...,C;C++;Java;JavaScript;TypeScript;VB.NET;HTML;CSS,MySQL;PostgreSQL,MongoDB;MySQL;PostgreSQL;SQLite;Microsoft Azur...,Android;iOS;Windows Phone,Gaming console;iOS;Raspberry Pi;Windows Deskto...,Angular;Cordova,Hadoop;Node.js;React;Xamarin,Windows,Somewhat interested,25 - 34 years old
71510,101567,Australia,"Yes, full-time","10,000 or more employees",No,Back-end developer,Male,Moderately satisfied,Between 1 and 2 years ago,140000,...,Java;JavaScript;Python;Ruby;TypeScript;Kotlin,MySQL;PostgreSQL;Amazon DynamoDB;Amazon Redshi...,Amazon DynamoDB,Amazon Echo;Android,Amazon Echo,Node.js;React;Spring,Spring,MacOS,Extremely interested,25 - 34 years old
71524,101584,United States,"Yes, full-time",20 to 99 employees,No,"Back-end developer;C-suite executive (CEO, CTO...",Male,Extremely satisfied,More than 4 years ago,165000,...,Clojure;R,Redis;SQL Server;PostgreSQL;Microsoft Azure (T...,Redis;PostgreSQL,AWS;Azure;Google Cloud Platform/App Engine;Lin...,Linux,Django;Node.js;React;TensorFlow;Torch/PyTorch,Django;React;TensorFlow;Torch/PyTorch,MacOS,Not at all interested,35 - 44 years old


#### Data Cleaning : DevType

In [111]:
Resume['DevType'].value_counts()

Full-stack developer                                                                                                                                                                                                                                    1111
Back-end developer;Front-end developer;Full-stack developer                                                                                                                                                                                              981
Back-end developer                                                                                                                                                                                                                                       914
Back-end developer;Full-stack developer                                                                                                                                                                                                          

Removing dev type category having less than 100 entries in dataset

In [112]:
Devtype_value= Resume['DevType'].value_counts()
Resume2= Resume[Resume['DevType'].isin(Devtype_value[Devtype_value>100].index)]
Resume2

Resume2['DevType'] = Resume2['DevType'].str.replace(';',',')
Resume2['DevType'] = Resume2[[Resume2.columns[5]]].values.tolist()
Resume2['DevType']

for index,data in Resume2.iterrows():
    empList=[]
    array_devtype= data['DevType']
    for dev in array_devtype:
        devsplit= dev.split(',')
    Resume2.at[index,'DevType']=devsplit 
        
Resume2.iloc[1]['DevType']

C:\Users\hp\AppData\Local\Temp/ipykernel_16684/1837419303.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Resume2['DevType'] = Resume2['DevType'].str.replace(';',',')
C:\Users\hp\AppData\Local\Temp/ipykernel_16684/1837419303.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Resume2['DevType'] = Resume2[[Resume2.columns[5]]].values.tolist()


['Mobile developer']

In [113]:
DevType_co = pd.Series(Counter(chain.from_iterable(Resume2.DevType.dropna())))
DevType_co

Back-end developer                              4050
Mobile developer                                 495
Desktop or enterprise applications developer     542
Front-end developer                             2415
Full-stack developer                            4117
DevOps specialist                                388
Designer                                         118
dtype: int64

#### Data Cleaning : Age

In [114]:
Resume2['Age'].value_counts()

25 - 34 years old     3567
18 - 24 years old     1053
35 - 44 years old      986
45 - 54 years old      204
55 - 64 years old       30
Under 18 years old       6
65 years or older        3
Name: Age, dtype: int64

#### Data Cleaning : Operating System

In [115]:
Resume2['OperatingSystem'].value_counts()

Windows        2773
MacOS          1925
Linux-based    1146
BSD/Unix          5
Name: OperatingSystem, dtype: int64

#### Data Cleaning : Location
Location entries having more than 50 resumes/respondents are considered

In [116]:
location= Resume2['Country'].value_counts()
location[location>50].sum()

4639

In [117]:
Resume3= Resume2[Resume2['Country'].isin(location[location>50].index)]
Resume3

,Respondent,Country,Student,CompanySize,Dependents,DevType,Gender,JobSatisfaction,LastNewJob,Salary,...,LanguageDesireNextYear,DatabaseWorkedWith,DatabaseDesireNextYear,PlatformWorkedWith,PlatformDesireNextYear,FrameworkWorkedWith,FrameworkDesireNextYear,OperatingSystem,HypotheticalTools1,Age
17,27,Sweden,No,10 to 19 employees,Yes,[Back-end developer],Male,Moderately satisfied,Between 1 and 2 years ago,32000,...,C#;F#;Haskell;SQL;Ocaml,SQL Server,Redis;SQL Server;PostgreSQL;Elasticsearch,Linux;Windows Desktop or Server,Linux;Windows Desktop or Server,.NET Core,.NET Core,Windows,Not at all interested,35 - 44 years old
24,39,India,No,100 to 499 employees,No,[Mobile developer],Female;Male,Moderately satisfied,Between 1 and 2 years ago,700000,...,Java;JavaScript;Swift;Kotlin,SQL Server;MySQL,MySQL,Android;iOS;Firebase,Android;iOS;Firebase,Angular;Cordova,Cordova,MacOS,Somewhat interested,25 - 34 years old
45,71,India,No,100 to 499 employees,Yes,"[Back-end developer, Desktop or enterprise app...",Male,Moderately satisfied,More than 4 years ago,35000,...,CoffeeScript;Go;JavaScript;Matlab;Perl;PHP;Pyt...,MySQL;MariaDB,MySQL;PostgreSQL;MariaDB;Google BigQuery;Micro...,WordPress,Amazon Echo;Android;Apple Watch or Apple TV;Ga...,Django,Angular;Django;Node.js;React;Cordova,Windows,Very interested,25 - 34 years old
46,72,United States,No,20 to 99 employees,No,"[Back-end developer, Full-stack developer]",Male,Slightly satisfied,Between 1 and 2 years ago,115000,...,Groovy;Haskell;Rust;Scala;Kotlin,Cassandra;MongoDB;Amazon DynamoDB,Cassandra;Apache HBase;Elasticsearch,Amazon Echo;AWS;Linux,Linux,Node.js;Spring,Hadoop;Spark,Linux-based,Somewhat interested,25 - 34 years old
111,161,United States,No,10 to 19 employees,No,[Back-end developer],Male,Extremely satisfied,Less than a year ago,120000,...,C++;Erlang;Go;R;SQL;Bash/Shell,Redis;PostgreSQL;Amazon DynamoDB,Cassandra;Redis;PostgreSQL;Amazon DynamoDB,Arduino;AWS;Linux;Mac OS;Raspberry Pi,Arduino;AWS;ESP8266;Linux;Mac OS;Raspberry Pi;...,Django;React,TensorFlow,MacOS,Somewhat interested,25 - 34 years old
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71470,101518,India,No,500 to 999 employees,No,"[Back-end developer, Full-stack developer]",Male,Moderately satisfied,Between 1 and 2 years ago,700000,...,Go;Rust,MongoDB;Redis;MySQL;PostgreSQL;Amazon DynamoDB...,Google BigQuery;Elasticsearch,Linux;WordPress,Amazon Echo,Node.js;React,Hadoop,Linux-based,Extremely interested,25 - 34 years old
71482,101532,United States,No,500 to 999 employees,Yes,"[Back-end developer, Desktop or enterprise app...",Male,Moderately satisfied,Between 2 and 4 years ago,135000,...,C#;Go;Python;SQL,SQL Server;MySQL;SQLite,"SQL Server;MySQL;Microsoft Azure (Tables, Cosm...",Windows Desktop or Server,Android;Linux;Raspberry Pi;Windows Desktop or ...,.NET Core;Node.js,.NET Core;Node.js,Windows,Somewhat interested,35 - 44 years old
71484,101535,Germany,No,20 to 99 employees,No,[Back-end developer],Male,Moderately satisfied,Less than a year ago,50000,...,Go;Java;JavaScript;Python;SQL;Bash/Shell,MySQL;SQLite;MariaDB,Cassandra;MongoDB;Redis;MySQL;SQLite;MariaDB;N...,Amazon Echo;AWS;Linux;Raspberry Pi,Amazon Echo;AWS;Google Home;Linux;Raspberry Pi...,Node.js;Spring,Node.js;Spring,Linux-based,Not at all interested,25 - 34 years old
71503,101558,Germany,No,20 to 99 employees,No,[Full-stack developer],Male,Slightly dissatisfied,Less than a year ago,50000,...,Ruby;Rust;Swift;HTML;CSS;Bash/Shell,MongoDB;Redis;MySQL;PostgreSQL;Microsoft Azure...,Redis;PostgreSQL;Elasticsearch,AWS;Azure;Heroku;iOS;Mac OS;SharePoint,Heroku;iOS;Mac OS,Angular;.NET Core;Cordova,React,MacOS,A little bit interested,25 - 34 years old


In [118]:
Country_count=Resume3['Country'].unique()
len(Country_count)

20

#### Data Cleaning : Gender
Categorising Gender into Male,Female and others

In [119]:
gender_value=Resume3.groupby('Gender')['Gender'].agg('count').sort_values(ascending=False)
print(gender_value)

Gender
Male                                                                    4274
Female                                                                   301
Non-binary, genderqueer, or gender non-conforming                         19
Female;Transgender                                                        10
Transgender                                                                9
Female;Non-binary, genderqueer, or gender non-conforming                   8
Male;Non-binary, genderqueer, or gender non-conforming                     8
Transgender;Non-binary, genderqueer, or gender non-conforming              4
Male;Transgender                                                           3
Female;Male                                                                2
Female;Transgender;Non-binary, genderqueer, or gender non-conforming       1
Name: Gender, dtype: int64


In [120]:
gender_less_than_40= gender_value[gender_value<40]
gender_less_than_40

Gender
Non-binary, genderqueer, or gender non-conforming                       19
Female;Transgender                                                      10
Transgender                                                              9
Female;Non-binary, genderqueer, or gender non-conforming                 8
Male;Non-binary, genderqueer, or gender non-conforming                   8
Transgender;Non-binary, genderqueer, or gender non-conforming            4
Male;Transgender                                                         3
Female;Male                                                              2
Female;Transgender;Non-binary, genderqueer, or gender non-conforming     1
Name: Gender, dtype: int64

In [121]:
Resume3.Gender= Resume3.Gender.apply(lambda x: 'other' if x in gender_less_than_40 else x)
len(Resume3.Gender.unique())

C:\Users\hp\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


3

### Data Cleaning : Skills
Combining 'Language' , 'Platform', 'Database' and 'Framework worked with' columns

In [122]:
# finding index no of Language Worked With column
index_no = Resume3.columns.get_loc('LanguageWorkedWith')
index_no

11

In [123]:
Resume3['LanguageWorkedWith'] = Resume3['LanguageWorkedWith'].str.replace(';',',')
Resume3['PlatformWorkedWith'] = Resume3['PlatformWorkedWith'].str.replace(';',',')
Resume3['DatabaseWorkedWith'] = Resume3['DatabaseWorkedWith'].str.replace(';',',')
Resume3['FrameworkWorkedWith'] = Resume3['FrameworkWorkedWith'].str.replace(';',',')

C:\Users\hp\AppData\Local\Temp/ipykernel_16684/2015081233.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Resume3['LanguageWorkedWith'] = Resume3['LanguageWorkedWith'].str.replace(';',',')
C:\Users\hp\AppData\Local\Temp/ipykernel_16684/2015081233.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Resume3['PlatformWorkedWith'] = Resume3['PlatformWorkedWith'].str.replace(';',',')
C:\Users\hp\AppData\Local\Temp/ipykernel_16684/2015081233.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy

In [124]:
Resume3['skills'] = Resume3[[Resume3.columns[11],Resume3.columns[13],Resume3.columns[15],Resume3.columns[17]]].values.tolist()
Resume3['skills']

C:\Users\hp\AppData\Local\Temp/ipykernel_16684/43760482.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Resume3['skills'] = Resume3[[Resume3.columns[11],Resume3.columns[13],Resume3.columns[15],Resume3.columns[17]]].values.tolist()


17       [C#,SQL,HTML,CSS,Bash/Shell, SQL Server, Linux...
24       [C,C++,Java,JavaScript,SQL,Swift,Kotlin, SQL S...
45       [JavaScript,PHP,Python,SQL,HTML,CSS, MySQL,Mar...
46       [Erlang,Go,Groovy,Java,JavaScript,Ruby,Scala,T...
111      [Go,JavaScript,Python,R,SQL,HTML,CSS,Bash/Shel...
                               ...                        
71470    [Assembly,C,JavaScript,PHP,SQL,HTML,CSS, Mongo...
71482    [C#,JavaScript,Perl,PHP,Python,SQL,CSS,Bash/Sh...
71484    [C++,Java,JavaScript,Python,SQL,HTML,CSS,Bash/...
71503    [C#,JavaScript,Ruby,Swift,TypeScript,HTML,CSS,...
71510    [Java,JavaScript,SQL,Bash/Shell, MySQL,Postgre...
Name: skills, Length: 4639, dtype: object

In [125]:
Resume3.iloc[0]['skills']

['C#,SQL,HTML,CSS,Bash/Shell',
 'SQL Server',
 'Linux,Windows Desktop or Server',
 '.NET Core']

Finding unique skills

In [126]:
for index,data in Resume3.iterrows():
    arryOfSkills=data['skills']
    megaskillslist=[]
    for skillStr in arryOfSkills:
        splitEachStr=skillStr.split(',')
        megaskillslist.extend(splitEachStr)
    
    for i in range (len(megaskillslist)):
        megaskillslist[i]= megaskillslist[i].lower()
    unique_list = (list(set(megaskillslist)))
    Resume3.at[index,'skills']=unique_list
Resume3.iloc[0]['skills']

['css',
 '.net core',
 'sql',
 'bash/shell',
 'c#',
 'sql server',
 'linux',
 'windows desktop or server',
 'html']

### Data Cleaning: Dependents

In [127]:
Resume3['Dependents'].value_counts()

No     3210
Yes    1429
Name: Dependents, dtype: int64

In [128]:
Resume3['Student'].value_counts()

No                4180
Yes, full-time     272
Yes, part-time     187
Name: Student, dtype: int64

### Data Cleaning: Salary
- Removing inconsitent data (comma)
- Converting (yearly,Weekly,Monthly) into consistent monthly salary 
- Binning salary in multiple intervals

In [129]:
Salary_count= Resume3['Salary'].value_counts()
type(Resume3['Salary'])
Salary_count.describe()

count    823.000000
mean       5.636695
std       13.561986
min        1.000000
25%        1.000000
50%        1.000000
75%        4.000000
max      123.000000
Name: Salary, dtype: float64

In [130]:
Resume3['SalaryType'].value_counts()

Yearly     3066
Monthly    1502
Weekly       71
Name: SalaryType, dtype: int64

In [131]:
Resume3['Salary']=Resume3['Salary'].str.replace(',','')
Resume3['Salary']

C:\Users\hp\AppData\Local\Temp/ipykernel_16684/3608720425.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Resume3['Salary']=Resume3['Salary'].str.replace(',','')


17        32000
24       700000
45        35000
46       115000
111      120000
          ...  
71470    700000
71482    135000
71484     50000
71503     50000
71510    140000
Name: Salary, Length: 4639, dtype: object

In [132]:
def salary_to_monthly(x,y):
    sal = x
    typ= y
    if (typ=="Yearly"):
        #print(typ)
        return (float(sal)/12)
    if (typ=="Weekly"):
        return (float(sal)*4)
    if (typ=="Monthly"):
        return (float(sal))    



In [133]:
Resume4= Resume3.copy()
# Salary conversion

#print(salary, salarytype)
for index,data in Resume4.iterrows():
    Resume4.at[index,'Salary'] = salary_to_monthly(Resume4.at[index,'Salary'],Resume4.at[index,'SalaryType'])

Resume4.head()


,Respondent,Country,Student,CompanySize,Dependents,DevType,Gender,JobSatisfaction,LastNewJob,Salary,...,DatabaseWorkedWith,DatabaseDesireNextYear,PlatformWorkedWith,PlatformDesireNextYear,FrameworkWorkedWith,FrameworkDesireNextYear,OperatingSystem,HypotheticalTools1,Age,skills
17,27,Sweden,No,10 to 19 employees,Yes,[Back-end developer],Male,Moderately satisfied,Between 1 and 2 years ago,32000.0,...,SQL Server,Redis;SQL Server;PostgreSQL;Elasticsearch,"Linux,Windows Desktop or Server",Linux;Windows Desktop or Server,.NET Core,.NET Core,Windows,Not at all interested,35 - 44 years old,"[css, .net core, sql, bash/shell, c#, sql serv..."
24,39,India,No,100 to 499 employees,No,[Mobile developer],other,Moderately satisfied,Between 1 and 2 years ago,58333.333333,...,"SQL Server,MySQL",MySQL,"Android,iOS,Firebase",Android;iOS;Firebase,"Angular,Cordova",Cordova,MacOS,Somewhat interested,25 - 34 years old,"[swift, android, mysql, firebase, ios, sql, ja..."
45,71,India,No,100 to 499 employees,Yes,"[Back-end developer, Desktop or enterprise app...",Male,Moderately satisfied,More than 4 years ago,35000.0,...,"MySQL,MariaDB",MySQL;PostgreSQL;MariaDB;Google BigQuery;Micro...,WordPress,Amazon Echo;Android;Apple Watch or Apple TV;Ga...,Django,Angular;Django;Node.js;React;Cordova,Windows,Very interested,25 - 34 years old,"[css, python, php, mysql, mariadb, sql, javasc..."
46,72,United States,No,20 to 99 employees,No,"[Back-end developer, Full-stack developer]",Male,Slightly satisfied,Between 1 and 2 years ago,9583.333333,...,"Cassandra,MongoDB,Amazon DynamoDB",Cassandra;Apache HBase;Elasticsearch,"Amazon Echo,AWS,Linux",Linux,"Node.js,Spring",Hadoop;Spark,Linux-based,Somewhat interested,25 - 34 years old,"[mongodb, scala, html, amazon echo, bash/shell..."
111,161,United States,No,10 to 19 employees,No,[Back-end developer],Male,Extremely satisfied,Less than a year ago,10000.0,...,"Redis,PostgreSQL,Amazon DynamoDB",Cassandra;Redis;PostgreSQL;Amazon DynamoDB,"Arduino,AWS,Linux,Mac OS,Raspberry Pi",Arduino;AWS;ESP8266;Linux;Mac OS;Raspberry Pi;...,"Django,React",TensorFlow,MacOS,Somewhat interested,25 - 34 years old,"[go, css, python, raspberry pi, react, sql, ja..."


In [134]:
Salary= Resume4['Salary']
print(min(Salary))
print(Salary.max())
print(sum(Salary))
print(sum(Salary)/len(Salary))

0.1
4300000.0
175979338.9950001
37934.75727419704


Binning into salary catergories

In [135]:
Salary_Category= pd.cut(Resume4.Salary, bins=[500,1500,5000,10000,16000,30000,45000,63000,79000,89000,100000], labels=['500-1500','1501-5000','5001-10000','10001-16000','16001-30000','30001-45000','45001-63000','63001-79000','79001-89000','89001-100000'] )
try:
    Resume4.insert(9,'Salary_group',Salary_Category)
except:
    print("Salary Group already exist!")

In [136]:
Resume4['Salary_group']
Resume4['Salary']

17            32000.0
24       58333.333333
45            35000.0
46        9583.333333
111           10000.0
             ...     
71470    58333.333333
71482         11250.0
71484     4166.666667
71503     4166.666667
71510    11666.666667
Name: Salary, Length: 4639, dtype: object

In [137]:
Resume4['Salary_group'].value_counts()

5001-10000      1444
1501-5000       1287
10001-16000      504
16001-30000      264
30001-45000      256
45001-63000      222
63001-79000      121
500-1500          78
89001-100000      67
79001-89000       46
Name: Salary_group, dtype: int64

In [138]:
Resume4['Salary_group'].isna().sum()
# drop na values i.e. remove outliers from salary
Resume5= Resume4.dropna()


In [194]:
analysis_data1= Resume4.groupby(['Age','Gender'])['skills']
analysis_data1

In [195]:
for x in analysis_data1:
    skill_count = pd.Series(Counter(chain.from_iterable(x[1].dropna()))).sort_values(ascending=False)
    print(x[0],skill_count)


('18 - 24 years old', 'Female') javascript                 47
css                        45
html                       44
node.js                    31
mysql                      31
                           ..
apple watch or apple tv     1
google home                 1
objective-c                 1
amazon echo                 1
vb.net                      1
Length: 78, dtype: int64
('18 - 24 years old', 'Male') javascript              693
html                    654
css                     636
sql                     495
mysql                   477
                       ... 
torch/pytorch             6
delphi/object pascal      5
gaming console            3
julia                     2
hack                      2
Length: 99, dtype: int64
('18 - 24 years old', 'other') css              13
javascript       12
html             12
node.js           8
linux             8
                 ..
groovy            1
elasticsearch     1
tensorflow        1
r                 1
raspberry pi      1

In [189]:
analysis_data2= Resume4.groupby(['Age','Gender'])['DevType']
analysis_data2

In [190]:
for x in analysis_data2:
    DevType_count = pd.Series(Counter(chain.from_iterable(x[1].dropna()))).sort_values(ascending=False)
    print(x[0],DevType_count)


('18 - 24 years old', 'Female') Back-end developer                              31
Full-stack developer                            31
Front-end developer                             24
Mobile developer                                 4
Desktop or enterprise applications developer     3
DevOps specialist                                2
dtype: int64
('18 - 24 years old', 'Male') Full-stack developer                            528
Back-end developer                              514
Front-end developer                             350
Mobile developer                                 79
Desktop or enterprise applications developer     53
DevOps specialist                                38
Designer                                         21
dtype: int64
('18 - 24 years old', 'other') Back-end developer      12
Full-stack developer    11
Front-end developer      6
DevOps specialist        1
Mobile developer         1
dtype: int64
('25 - 34 years old', 'Female') Full-stack developer           

### Build Graph on Neo4j


In [ ]:
Resume5.head()

,Respondent,Country,Student,CompanySize,Dependents,DevType,Gender,JobSatisfaction,LastNewJob,Salary_group,...,DatabaseWorkedWith,DatabaseDesireNextYear,PlatformWorkedWith,PlatformDesireNextYear,FrameworkWorkedWith,FrameworkDesireNextYear,OperatingSystem,HypotheticalTools1,Age,skills
17,27,Sweden,No,10 to 19 employees,Yes,[Back-end developer],Male,Moderately satisfied,Between 1 and 2 years ago,30001-45000,...,SQL Server,Redis;SQL Server;PostgreSQL;Elasticsearch,"Linux,Windows Desktop or Server",Linux;Windows Desktop or Server,.NET Core,.NET Core,Windows,Not at all interested,35 - 44 years old,"[css, .net core, sql, bash/shell, c#, sql serv..."
24,39,India,No,100 to 499 employees,No,[Mobile developer],other,Moderately satisfied,Between 1 and 2 years ago,45001-63000,...,"SQL Server,MySQL",MySQL,"Android,iOS,Firebase",Android;iOS;Firebase,"Angular,Cordova",Cordova,MacOS,Somewhat interested,25 - 34 years old,"[swift, android, mysql, firebase, ios, sql, ja..."
45,71,India,No,100 to 499 employees,Yes,"[Back-end developer, Desktop or enterprise app...",Male,Moderately satisfied,More than 4 years ago,30001-45000,...,"MySQL,MariaDB",MySQL;PostgreSQL;MariaDB;Google BigQuery;Micro...,WordPress,Amazon Echo;Android;Apple Watch or Apple TV;Ga...,Django,Angular;Django;Node.js;React;Cordova,Windows,Very interested,25 - 34 years old,"[css, python, php, mysql, mariadb, sql, javasc..."
46,72,United States,No,20 to 99 employees,No,"[Back-end developer, Full-stack developer]",Male,Slightly satisfied,Between 1 and 2 years ago,5001-10000,...,"Cassandra,MongoDB,Amazon DynamoDB",Cassandra;Apache HBase;Elasticsearch,"Amazon Echo,AWS,Linux",Linux,"Node.js,Spring",Hadoop;Spark,Linux-based,Somewhat interested,25 - 34 years old,"[mongodb, scala, html, amazon echo, bash/shell..."
111,161,United States,No,10 to 19 employees,No,[Back-end developer],Male,Extremely satisfied,Less than a year ago,5001-10000,...,"Redis,PostgreSQL,Amazon DynamoDB",Cassandra;Redis;PostgreSQL;Amazon DynamoDB,"Arduino,AWS,Linux,Mac OS,Raspberry Pi",Arduino;AWS;ESP8266;Linux;Mac OS;Raspberry Pi;...,"Django,React",TensorFlow,MacOS,Somewhat interested,25 - 34 years old,"[go, css, python, raspberry pi, react, sql, ja..."


In [ ]:
from neo4j import GraphDatabase
import logging
from neo4j.exceptions import ServiceUnavailable

class App:

    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        # Don't forget to close the driver connection when you are finished with it
        self.driver.close()

    def create_friendship(self, person1_name,skillVal,gednerVal,osNameVal,countryNameVal,saleryVal,devTypeVal,ageVal,dependsVal):
        with self.driver.session() as session:
            # Write transactions allow the driver to handle retries and transient errors
            result = session.write_transaction(
                self._create_and_return_friendship, person1_name,skillVal,gednerVal,osNameVal,countryNameVal,saleryVal,devTypeVal,ageVal,dependsVal)
            for row in result:
                print("Created Node between: {p1}, {p2}".format(p1=row['p1'], p2=row['p2']))

    @staticmethod
    def _create_and_return_friendship(tx, person1_name, skillVal ,gednerVal,osNameVal,countryNameVal,saleryVal,devTypeVal,ageVal,dependsVal):
        # To learn more about the Cypher syntax, see https://neo4j.com/docs/cypher-manual/current/
        # The Reference Card is also a good resource for keywords https://neo4j.com/docs/cypher-refcard/current/
        query = (
            "MERGE (p1:ResumeId { name: $person1_name }) "
            "MERGE (p2:Gender { name: $gednerVal }) "
            "MERGE (p3:OperatingSystem { name: $osNameVal }) "
            "MERGE (p4:Country { name: $countryNameVal }) "
            "MERGE (p5:Salery { name: $saleryVal }) "
            "MERGE (p6:DevType { name: $devTypeVal }) "
            "MERGE (p7:Age { name: $ageVal }) "
            "MERGE (p8:Dependents { name: $dependsVal }) "
            "CREATE (p1)-[:isA]->(p2) "
            "CREATE (p1)-[:hasOs]->(p3) "
            "CREATE (p1)-[:livesIn]->(p4) "
            "CREATE (p1)-[:earns]->(p5) "
            "CREATE (p1)-[:worksOn]->(p6) "
            "CREATE (p1)-[:liesIn]->(p7) "
            "CREATE (p1)-[:has]->(p8) "
            "RETURN p1,p2,p3,p4,p5,p6,p7,p8"
        )
        result = tx.run(query, person1_name=person1_name,gednerVal=gednerVal,osNameVal=osNameVal,countryNameVal=countryNameVal,saleryVal=saleryVal,devTypeVal=devTypeVal,ageVal=ageVal,dependsVal=dependsVal )
        try:
            return [{"p1": row["p1"]["name"], "p2": row["p2"]["name"]}
                    for row in result]
        # Capture any errors along with the query and data for traceability
        except ServiceUnavailable as exception:
            logging.error("{query} raised an error: \n {exception}".format(
                query=query, exception=exception))
            raise

    def find_person(self, person_name):
        with self.driver.session() as session:
            result = session.read_transaction(self._find_and_return_person, person_name)
            for row in result:
                print("Found person: {row}".format(row=row))

    @staticmethod
    def _find_and_return_person(tx, person_name):
        query = (
            "MATCH (p:Person) "
            "WHERE p.name = $person_name "
            "RETURN p.name AS name"
        )
        result = tx.run(query, person_name=person_name)
        return [row["name"] for row in result]


if __name__ == "__main__":
    # Aura queries use an encrypted connection using the "neo4j+s" URI scheme
    uri = "neo4j://localhost:7687"
    user = "neo4j"
    password = "amit"
    app = App(uri, user, password)
    for index,rows in Resume5.iterrows():
        resId=rows['Respondent'] 
        skillVal=rows['skills']
        gednerVal=rows['Gender']
        osNameVal=rows['OperatingSystem']
        countryNameVal=rows['Country']
        saleryVal=rows['Salary']
        devTypeVal=rows['DevType']
        ageVal=rows['Age']
        dependsVal=rows['Dependents']
        # print(resId,gednerVal)
        app.create_friendship(resId,skillVal,gednerVal,osNameVal,countryNameVal,saleryVal,devTypeVal,ageVal,dependsVal)
    app.find_person("India")
    app.close()

Unable to retrieve routing information
Transaction failed and will be retried in 1.0805415941429521s (Unable to retrieve routing information)
Unable to retrieve routing information
Transaction failed and will be retried in 2.351706948093435s (Unable to retrieve routing information)
Unable to retrieve routing information
Transaction failed and will be retried in 4.457965694649929s (Unable to retrieve routing information)
Unable to retrieve routing information
Transaction failed and will be retried in 8.843086350464347s (Unable to retrieve routing information)
Unable to retrieve routing information


ServiceUnavailable: Unable to retrieve routing information